# Sprint clase 3

## Regresión lineal múltiple del dataset **Medical Cost Personal**

En el sprint de la clase 2, tenemos el análisis exploratorio de los datos,
en este notebook vamos a realizar las fases de preprocesamiento, entrenamiento y evaluación.

Vamos a plantear diferentes hipótesis para predecir los gastos a partir de los datos de los usuarios.

Para evaluar los resultados obtenidos, vamos a utilizar el MSE (error cuadrático medio) como métrica.



### Importación de librerías que vamos a utilizar

In [69]:
# Por llevar una ordenación, me gusta en primer lugar poner los imports de los paquetes que utilizaremos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

### Importamos el dataset

In [70]:
# Trabajo con VsCode, en un entorno virtual con conda, y dispongo del dataset en el mismo directorio que el notebook
df_medical_cost = pd.read_csv('insurance.csv')

Comprobamos que los datos se han importado correctamente

In [71]:
df_medical_cost.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [72]:
df_medical_cost.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

Tal como se ha comentado anteriormente, el análisis exploratorio está en el notebook del sprint de la clase 2.

Aquí vamos a realizar el preprocesamiento de los datos, el entrenamiento y la evaluación.

### Hipótesis 1: Predicción de los gastos a partir de los campos numéricos

In [73]:
# Voy a crear dos listas, una con las columnas numéricas y otra con las categóricas
numerical_cols = df_medical_cost.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df_medical_cost.select_dtypes(include=object).columns.tolist()

print(f'Columnas numéricas: {numerical_cols}')
print(f'Columnas categóricas: {categorical_cols}')

Columnas numéricas: ['age', 'bmi', 'children', 'charges']
Columnas categóricas: ['sex', 'smoker', 'region']


Creamos el dataframe con los campos numéricos.

En primer lugar eliminamos las columnas categóricas:

In [74]:
df_medical_cost_hipot1 = df_medical_cost.drop(categorical_cols, axis=1)

In [75]:
# Comprobamos que no hay columnas categóricas
df_medical_cost_hipot1.head()

,age,bmi,children,charges
0,19,27.900,0,16884.92400
1,18,33.770,1,1725.55230
2,28,33.000,3,4449.46200
3,33,22.705,0,21984.47061
4,32,28.880,0,3866.85520


Creamos los dataframe con las variables independientes y la variable dependiente:

In [76]:
X_hipot1 = df_medical_cost_hipot1.drop('charges', axis=1)
y_hipot1 = df_medical_cost_hipot1['charges']

Creamos los conjuntos de entrenamiento y test, y creamos la regresión lineal:

In [77]:
X_hipot1_train, X_hipot1_test, y_hipot1_train, y_hipot1_test = train_test_split(X_hipot1, y_hipot1, test_size=0.2, random_state=42)

linear_regression_1 = LinearRegression()
linear_regression_1.fit(X_hipot1_train, y_hipot1_train)

LinearRegression()

Calculamos el MSE:

Creo una función que me servirá par esta y las siguientes hipotesis:


In [78]:
def calculate_mse(X_train, X_test, y_train, y_test, model):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    mse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    mse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(f'MSE train: {mse_train}')
    print(f'MSE test: {mse_test}')

In [79]:
print('Hipótesis 1')
calculate_mse(X_hipot1_train, X_hipot1_test, y_hipot1_train, y_hipot1_test, linear_regression_1)

Hipótesis 1
MSE train: 11334.681601852764
MSE test: 11454.315153980095


Los valores obtenidos son altos, vamos a probar transformando las variables categoricas en numéricas.

### Hipótesis 2: Predicción de los gastos transformando las variables categóricas en numéricas

Creamos el dataframe con las variables categóricas transformadas en numéricas:

In [80]:
df_medical_cost_hipot2 = pd.get_dummies(data=df_medical_cost, 
                                        columns=categorical_cols, 
                                        prefix='OHE',
                                        prefix_sep='_',
                                        drop_first=True,
                                        dtype=np.int8)

In [81]:
# Comprobamos que se han convertido las columnas categóricas correctamente
df_medical_cost_hipot2.head()

,age,bmi,children,charges,OHE_male,OHE_yes,OHE_northwest,OHE_southeast,OHE_southwest
0,19,27.900,0,16884.92400,0,1,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,1,0,0


Creamos los dataframe con las variables independientes y la variable dependiente, repetimos el proceso como anteriormente:

In [82]:
X_hipot2 = df_medical_cost_hipot2.drop('charges', axis=1)
y_hipot2 = df_medical_cost_hipot2['charges']

In [83]:
X_hipot2_train, X_hipot2_test, y_hipot2_train, y_hipot2_test = train_test_split(X_hipot2, y_hipot2, test_size=0.2, random_state=42)
linear_regression_2 = LinearRegression()
linear_regression_2.fit(X_hipot2_train, y_hipot2_train)

LinearRegression()

Calculamos el MSE para la hipótesis 2:

In [84]:
print('Hipótesis 2')
calculate_mse(X_hipot2_train, X_hipot2_test, y_hipot2_train, y_hipot2_test, linear_regression_2)

Hipótesis 2
MSE train: 6105.545160099847
MSE test: 5796.284659276272


Hemos mejorado un poco, pero sigue siendo alto. Vamos a probar con la hipótesis 3.

### Hipótesis 3: Predicción de los gastos normalizando la variable objetivo

Vamos a normalizar la variable `charges` para ver si mejora el resultado.

In [85]:
# Hacemos el one_hote_encoding de las columnas categóricas
df_medical_cost_hipot3 = pd.get_dummies(data=df_medical_cost, 
                                        columns=categorical_cols, 
                                        prefix='OHE',
                                        prefix_sep='_',
                                        drop_first=True,
                                        dtype=np.int8)

In [86]:
X_hipot3 = df_medical_cost_hipot3.drop('charges', axis=1)
y_hipot3 = df_medical_cost_hipot3['charges']

Escalamos la variable `charges`:

In [87]:
scaler = StandardScaler()

y_hipot3 = scaler.fit_transform(y_hipot3.values.reshape(-1, 1))
y_hipot3 = y_hipot3.ravel()

In [88]:
X_hipot3_train, X_hipot3_test, y_hipot3_train, y_hipot3_test = train_test_split(X_hipot3, y_hipot3, test_size=0.2, random_state=42)
linear_regression_3 = LinearRegression()
linear_regression_3.fit(X_hipot3_train, y_hipot3_train)

LinearRegression()

In [89]:
print('Hipótesis 3')
calculate_mse(X_hipot3_train, X_hipot3_test, y_hipot3_train, y_hipot3_test, linear_regression_3)

Hipótesis 3
MSE train: 0.5043618799598396
MSE test: 0.47881474149757175


Aquí el resultado lo podemos dar por bastante bueno, pero vamos a por una hipótesis más.

### Hipótesis 4: Predicción de los gastos normalizando la variable objetivo y `bmi`

Vamos a normalizar la variable `bmi`, además de `charges` para ver si mejora el resultado.

In [90]:
# Hacemos el one_hote_encoding de las columnas categóricas

df_medical_cost_hipot4 = pd.get_dummies(data=df_medical_cost, 
                                        columns=categorical_cols, 
                                        prefix='OHE',
                                        prefix_sep='_',
                                        drop_first=True,
                                        dtype=np.int8)

In [91]:
scaler = MaxAbsScaler()
cols_to_scale = ['bmi', 'charges']
df_medical_cost_hipot4[cols_to_scale] = scaler.fit_transform(df_medical_cost_hipot4[cols_to_scale])

In [92]:
df_medical_cost_hipot4.head()

,age,bmi,children,charges,OHE_male,OHE_yes,OHE_northwest,OHE_southeast,OHE_southwest
0,19,0.525127,0,0.264777,0,1,0,0,1
1,18,0.635611,1,0.027059,1,0,0,1,0
2,28,0.621118,3,0.069773,1,0,0,1,0
3,33,0.427348,0,0.344744,1,0,1,0,0
4,32,0.543572,0,0.060637,1,0,1,0,0


In [93]:
X_hipot4 = df_medical_cost_hipot4.drop('charges', axis=1)
y_hipot4 = df_medical_cost_hipot4['charges']

In [94]:
X_hipot4_train, X_hipot4_test, y_hipot4_train, y_hipot4_test = train_test_split(X_hipot4, y_hipot4, test_size=0.2, random_state=42)
linear_regression_4 = LinearRegression()
linear_regression_4.fit(X_hipot4_train, y_hipot4_train)

LinearRegression()

In [95]:
print('Hipótesis 4')
calculate_mse(X_hipot4_train, X_hipot4_test, y_hipot4_train, y_hipot4_test, linear_regression_4)

Hipótesis 4
MSE train: 0.09574257772179953
MSE test: 0.09089298661077425


Escalando la variable `bmi` tenemos un mejor resultado, por lo tanto podemos dar el modelo por bueno.
